real-time data implementation


First open OpenBCI and select LSL stream and place the electrode at the position of interest.


Then the electrical signal data will be analyzed by the model, and a prediction number will display every 3 seconds.

The result will be sent to the robot-car. 

In [12]:
import pyxdf
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter, filtfilt
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from pylsl import StreamInlet, resolve_stream
import time
import requests
model = load_model('model3s.keras')
# Initialize LSL connection

# Obtaining EEG data
def butter_bandpass(lowcut, highcut, fs, order=6):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data, padlen=25)  # Adjust padlen to be smaller than the default
    return y

# Sample rate and desired cutoff frequencies (in Hz)
fs = 250  # Sample rate
lowcut = 8  # Lower cutoff frequency
highcut = 16  # Higher cutoff frequency

#Function to compute FFT
def compute_fft(data, fs):
    n = len(data)
    fft_data = np.fft.fft(data) / n
    freqs = np.fft.fftfreq(n, 1 / fs)
    return freqs[:n // 2], np.abs(fft_data)[:n // 2]


def everysecond():
    
    valuelist = []
    #tick = time.time()
    while len(valuelist) < 750:
        sample, timestamps = inlet.pull_sample()
        if timestamps:
            valuelist.append(sample[1])  # Store the sample with its timestamp
  
    data = valuelist
    filtered_data = butter_bandpass_filter(data, lowcut, highcut, fs, order=6)
    freqs_filtered, fft_filtered = compute_fft(data, fs)
    # Update the time variables for the next iteration
    lower_bound=np.where(freqs_filtered==8)[0][0]
    higher_bound=np.where(freqs_filtered==16)[0][0]+1
    feature=fft_filtered[lower_bound:higher_bound]#only extract the power info from freq 8-16Hz
    feature_norm=feature/max(feature)
    example_list = feature_norm
    input_data = np.array(example_list).reshape(25, 1)
    input_data = input_data.reshape((1, 25, 1))
    prediction=model.predict(input_data)
    print(prediction) 
    print(np.argmax(prediction))
    requests.post('http://your.serverIP.should.be.here/in',json={"value":str(np.argmax(prediction)+1)})



In [19]:
stream_type = "EEG"
#print("Looking for an", stream_type, "stream...")
streams = resolve_stream("type", stream_type)
#print("Connecting to an", stream_type, "stream...")
inlet = StreamInlet(streams[0])
results = []
while True:
    everysecond()
    

[0.76938776 1.         0.47399236 0.21338011 0.63272157 0.11979739
 0.42441034 0.19280893 0.55849327 0.10526408 0.54605997 0.535477
 0.65068822 0.46497903 0.3394504  0.26688288 0.84732907 0.78585201
 0.48439639 0.46563101 0.44897032 0.28713933 0.06424796 0.35646196
 0.36229413]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[[0.01469463 0.00970109 0.97560424]]
2
[2]
[0.7818569  1.         0.30876793 0.3041798  0.77453102 0.41753964
 0.45105295 0.48304629 0.54350757 0.4315765  0.5815472  0.97268514
 0.44649861 0.8452962  0.60050129 0.42852528 0.87268744 0.83382683
 0.28910655 0.3797228  0.1799861  0.11792122 0.3974464  0.46945582
 0.14548471]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
[[0.01727108 0.00774751 0.97498137]]
2
[2, 2]
[0.17681959 0.84553701 0.07231665 0.55410725 0.63573725 0.22859324
 0.9986538  0.42618215 0.47590627 0.81822184 0.1924778  0.55548803
 0.46971085 0.03985128 0.4253073  1.         0.27362263 0.77063045
 0.58527844 0.23335776 0.36006842 0.85553866 0.40165741 0.41919436
 0.1906

KeyboardInterrupt: 